## ZTF
In this notebook, we run THOR on two weeks of ZTF alerts. We select test orbits from the catalog of known objects (MPCORB.DAT).

Data and results files for this notebook may be downloaded [here](https://dirac.astro.washington.edu/~moeyensj/projects/thor/paper1/).

In [1]:
%load_ext autoreload
%autoreload 2

import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set(font_scale=1.2, context="paper", style="ticks")
sns.set_palette("viridis")

from astropy.time import Time

%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

In [2]:
import thor

from thor import __version__
print("THOR version: {}".format(__version__))

THOR version: 1.1.dev199+g1c54766.d20210401


In [3]:
DATA_DIR = "/mnt/data/projects/thor/thor_data/ztf"

preprocessed_observations = pd.read_csv(
    "/mnt/data/projects/thor/thor_data/ztf/preprocessed_observations.csv",
    index_col=False,
    dtype={
        "obs_id" : str,
    }
)

In [4]:
preprocessed_observations.head(10)

,obs_id,mjd_utc,RA_deg,Dec_deg,RA_sigma_deg,Dec_sigma_deg,observatory_code
0,610130484415010015,58364.130486,255.347544,-23.059466,0.000028,0.000028,I41
1,610130481215010007,58364.130486,255.077637,-26.542553,0.000028,0.000028,I41
2,610130481215015021,58364.130486,254.708502,-26.721381,0.000028,0.000028,I41
3,610130483515015056,58364.130486,261.287334,-24.006969,0.000028,0.000028,I41
4,610130483515015069,58364.130486,261.062061,-23.963993,0.000028,0.000028,I41
5,610130481715015012,58364.130486,260.863805,-24.507448,0.000028,0.000028,I41
6,610130481715010004,58364.130486,260.598060,-24.383790,0.000028,0.000028,I41
7,610130481215015013,58364.130486,254.799074,-26.510711,0.000028,0.000028,I41
8,610130483515015052,58364.130486,261.668127,-23.836318,0.000028,0.000028,I41
9,610130483915015049,58364.130486,258.969586,-23.671591,0.000028,0.000028,I41


In [5]:
# Read orbits file (MPCORB in OORB format from 2018)

orbits = pd.read_csv(
    "/mnt/data/projects/thor/thor_data/ztf/MPCORB_20181106_ZTF_keplerian.orb", 
    delim_whitespace=True, 
    skiprows=4,
    names=["designation", "a_au", "e", "i_deg", "ascNode_deg", "argPeri_deg", "meanAnom_deg", "epoch_mjd_tt", "H", "G"],
    low_memory=False
)

In [6]:
simulated_ephemeris = pd.read_csv(
    "/mnt/data/projects/thor/thor_data/ztf/MPCORB_20181106_ZTF.eph",
    delim_whitespace=True,
    header=0,
    low_memory=False
)
simulated_ephemeris.rename(
    columns={
        "#Designation" : "designation",
        "RA" : "RA_deg",
        "Dec" : "Dec_deg",
        "MJD_UTC/UT1" : "exp_mjd",
        "r" : "r_au",
        "HEclObj_X" : "HEclObj_X_au",
        "HEclObj_Y" : "HEclObj_Y_au",
        "HEclObj_Z" : "HEclObj_Z_au",
        "HEclObj_dX/dt" : "HEclObj_dX/dt_au_p_day",
        "HEclObj_dY/dt" : "HEclObj_dY/dt_au_p_day",
        "HEclObj_dZ/dt" : "HEclObj_dZ/dt_au_p_day",
    },
    inplace=True
)

In [7]:
size = 15
ras = np.arange(0, 360 + size, size)
decs = np.arange(-90, 90 + size, size)

In [8]:
from thor import findAverageOrbits

average_orbits_list = []

patch_number = 0
for ra_i, ra_f in zip(ras[:-1], ras[1:]):
    for dec_i, dec_f in zip(decs[:-1], decs[1:]):
        # See if  there are any observations in the patch
        observations_in_patch = preprocessed_observations[
            (preprocessed_observations["RA_deg"] >= ra_i) 
            & (preprocessed_observations["RA_deg"] < ra_f)
            & (preprocessed_observations["Dec_deg"] < dec_f)
            & (preprocessed_observations["Dec_deg"] >= dec_i)
        ].copy()
        if len(observations_in_patch) > 0:
            # Find time of first set of observations in patch (need to propagate orbit to that time)
            exp_mjd = observations_in_patch["mjd_utc"].min()
            
            simulated_ephemeris_mask = (
                (simulated_ephemeris["RA_deg"] >= ra_i) 
                & (simulated_ephemeris["RA_deg"] < ra_f)
                & (simulated_ephemeris["Dec_deg"] < dec_f)
                & (simulated_ephemeris["Dec_deg"] >= dec_i)
            )
            
            average_orbits_hun1_patch = findAverageOrbits(
                simulated_ephemeris[simulated_ephemeris_mask],
                orbits[(orbits["a_au"] < 2.06) & (orbits["a_au"] >= 1.7) & (orbits["e"] <= 0.1)],
                element_type="keplerian",
                d_values=[1.7, 2.06]
            )
            average_orbits_hun2_patch = findAverageOrbits(
                simulated_ephemeris[simulated_ephemeris_mask],
                orbits[(orbits["a_au"] < 2.06) & (orbits["a_au"] >= 1.7) & (orbits["e"] > 0.1) & (orbits["e"] <= 0.2)],
                element_type="keplerian",
                d_values=[1.7, 2.06]
            )
            average_orbits_hun3_patch = findAverageOrbits(
                simulated_ephemeris[simulated_ephemeris_mask],
                orbits[(orbits["a_au"] < 2.06) & (orbits["a_au"] >= 1.7) & (orbits["e"] > 0.2) & (orbits["e"] <= 0.4)],
                element_type="keplerian",
                d_values=[1.7, 2.06]
            )

            average_orbits_patch = findAverageOrbits(
                simulated_ephemeris[simulated_ephemeris_mask],
                orbits[(orbits["e"] < 0.5)].reset_index(drop=True),
                element_type="keplerian",
                d_values=[2.06, 2.5, 2.82, 2.95, 3.27, 5.0, 50.0],
            )
            average_orbits_patch = pd.concat(
                [
                    average_orbits_hun1_patch, 
                    average_orbits_hun2_patch, 
                    average_orbits_hun3_patch, 
                    average_orbits_patch
                ], 
                ignore_index=True
            )
            average_orbits_patch.loc[:, "exp_mjd_start"] = exp_mjd 
            average_orbits_patch.insert(0, "patch_number", patch_number)
            average_orbits_list.append(average_orbits_patch)
            
            patch_number += 1
                                        
average_orbits = pd.concat(average_orbits_list)
average_orbits.sort_values(by=["patch_number", "a_au"], inplace=True)
average_orbits["orbit_id"] = np.arange(1, len(average_orbits) + 1)
average_orbits.reset_index(inplace=True, drop=True)

orbits["i_rad"] = np.radians(orbits["i_deg"])
orbits["ascNode_rad"] = np.radians(orbits["ascNode_deg"])
orbits["argPeri_rad"] = np.radians(orbits["argPeri_deg"])
orbits["meanAnom_rad"] = np.radians(orbits["meanAnom_deg"])

average_orbits = average_orbits.drop(index=average_orbits[average_orbits["designation"].isna()].index)
average_orbits.reset_index(
    inplace=True,
    drop=True
)

In [9]:
from astropy.time import Time
from thor.orbits import Orbits

average_orbits["epoch"] =  Time(average_orbits["exp_mjd"].values, scale="utc", format="mjd").tdb.mjd
average_orbits.rename(
    columns={
        ""
        "HEclObj_X_au" : "x",
        "HEclObj_Y_au" : "y",
        "HEclObj_Z_au" : "z",
        "HEclObj_dX/dt_au_p_day" : "vx",
        "HEclObj_dY/dt_au_p_day" : "vy",
        "HEclObj_dZ/dt_au_p_day" : "vz",
    },
    inplace=True
)

In [10]:
average_orbits

,patch_number,orbit_id,exp_mjd,x,y,z,vx,vy,vz,RA_deg,Dec_deg,r_au,a_au,i_deg,e,designation,exp_mjd_start,epoch
0,0,1,58364.130486,2.033718,-0.409943,-0.615010,0.001833,0.009813,0.003433,9.435607,-28.329819,2.163863,1.825904,24.27121,0.212059,16465,58366.335544,58364.131287
1,0,2,58364.130486,1.730103,-0.472806,-0.406931,0.004502,0.011699,0.003409,2.910473,-28.627382,1.839129,1.933515,20.46270,0.060869,K12F30O,58366.335544,58364.131287
2,0,3,58364.130486,1.975724,-0.358711,-0.568762,0.001647,0.010978,0.003013,11.501128,-26.830805,2.087019,1.956271,21.12614,0.120002,K10V50B,58366.335544,58364.131287
3,0,4,58364.130486,1.848637,-0.437143,-0.398335,0.003218,0.012930,0.002125,4.388285,-24.159871,1.940933,2.408633,14.83891,0.195266,11076,58366.335544,58364.131287
4,0,5,58364.130486,2.077365,-0.477214,-0.545286,0.002097,0.012240,0.000170,4.597215,-26.140055,2.200117,2.578931,14.35001,0.157613,e6197,58366.335544,58364.131287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,155,1402,58364.130486,4.116486,0.151388,1.861499,0.001066,0.006243,0.001031,354.767668,30.901130,4.520349,3.297043,24.58842,0.437700,K10K72Q,58364.249317,58364.131287
817,156,1405,58364.130486,1.547406,-0.159075,0.579829,0.000035,0.014029,0.001324,353.790586,45.131339,1.660112,1.873678,21.57870,0.128759,A5246,58364.249317,58364.131287
818,156,1406,58364.130486,1.450298,-0.147537,0.547233,0.004407,0.015055,-0.001543,355.264036,49.042734,1.557111,2.248037,22.38929,0.333293,K11L17F,58364.249317,58364.131287
819,156,1407,58364.130486,1.683562,-0.027564,0.938848,0.000326,0.013666,0.001616,353.224870,53.141855,1.927843,2.517908,29.64501,0.242378,K10M02Q,58364.249317,58364.131287


In [11]:
test_orbits = Orbits.from_df(
    average_orbits
)
orbit_file = "/mnt/data/projects/thor/thor_results/ztf/v1.1/test_orbits.csv"
if not os.path.exists(orbit_file):
    test_orbits.to_csv(orbit_file)

In [12]:
from thor.orbits import Orbits

# Patches in chunks of 5 were submitted to Hyak, we now combine each patch's recovered orbits
RUN_DIR = "/mnt/data/projects/thor/thor_results/ztf/v1.1/run4/"

patch_orbits = []
patch_orbit_members = []

contents = sorted(glob.glob(os.path.join(RUN_DIR, "patch_*")))
for c in contents:
    if os.path.isdir(c):
        patch_orbits_i = Orbits.from_csv(
            os.path.join(c, "recovered_orbits.csv")
        )
        patch_orbit_members_i = pd.read_csv(
            os.path.join(c, "recovered_orbit_members.csv"),
            index_col=False,
            dtype={
                "obs_id" : str
            }
        )
        patch_orbits.append(
            patch_orbits_i.to_df(include_units=False)
        )
        patch_orbit_members.append(patch_orbit_members_i)
        
        
patch_orbits = pd.concat(
    patch_orbits,
    ignore_index=True
)

patch_orbit_members = pd.concat(
    patch_orbit_members,
    ignore_index=True
)

In [13]:
from thor.utils import removeDuplicateLinkages
from thor.utils import removeDuplicateObservations
from thor.utils import sortLinkages

recovered_orbits, recovered_orbit_members = removeDuplicateLinkages(
    patch_orbits, 
    patch_orbit_members
)
recovered_orbits, recovered_orbit_members = removeDuplicateObservations(
    recovered_orbits, 
    recovered_orbit_members
)
recovered_orbits, recovered_orbit_members = sortLinkages(
    recovered_orbits, 
    recovered_orbit_members, 
    preprocessed_observations
)

In [14]:
from thor.orbits import differentialCorrection

recovered_orbits, recovered_orbit_members = differentialCorrection(
    recovered_orbits,
    recovered_orbit_members,
    preprocessed_observations, 
    min_obs=5,
    min_arc_length=1.0,
    rchi2_threshold=10,
    contamination_percentage=0.0,
    delta=1e-8,
    max_iter=10,
    method="central",
    fit_epoch=False,
    threads=60,
    backend="PYOORB", 
)

19:53:17.539 [INFO] thor.orbits.od - Running differential correction...
19:59:07.152 [INFO] thor.orbits.od - Differentially corrected 21723 orbits.
19:59:07.153 [INFO] thor.orbits.od - Differential correction completed in 349.612 seconds.


In [15]:
Orbits.from_df(recovered_orbits).to_csv(
    os.path.join(RUN_DIR, "recovered_orbits.csv")
)

recovered_orbit_members.to_csv(
    os.path.join(RUN_DIR, "recovered_orbit_members.csv"),
    index=False
)